In [ ]:
import os
from fastai.data.external import untar_data, URLs


import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor

import numpy as np

import random
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline


Скачиваем датасет CIFAR10

In [ ]:


dataset = torchvision.datasets.CIFAR10(root='./data', download=True)

Files already downloaded and verified


In [ ]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform)

batch_size = 100
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True, num_workers=2, pin_memory=True)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size * 2, num_workers=2, pin_memory=True)

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)


device = get_default_device()
print(device)

trainloader = DeviceDataLoader(trainloader, device)
testloader = DeviceDataLoader(testloader, device)

cuda


In [ ]:

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [ ]:
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        acc = accuracy(out, labels)  
        return loss,acc
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, train_acc: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}, last_lr: {:.5f}".format(
            epoch+1, result['train_loss'], result['train_accuracy'], result['val_loss'], result['val_acc'], result['lrs'][-1]))

Добавим 4 встроенные модели библиотеки pytorch

In [ ]:
alexNet = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
alexNet.eval()

resNet34 = torchvision.models.resnet34()
resNet34.eval()

resNext50 = torchvision.models.resnext50_32x4d()
resNext50.eval()

from torchinfo import summary

summary(alexNet, input_size=(batch_size, 3, 227, 227))
# summary(resNet34, input_size=(batch_size, 3, 227, 227))
# summary(resNext50, input_size=(batch_size, 3, 227, 227))

Using cache found in C:\Users\Otche/.cache\torch\hub\pytorch_vision_v0.10.0
c:\Users\Otche\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Otche\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Layer (type:depth-idx)                   Output Shape              Param #
AlexNet                                  [400, 1000]               --
├─Sequential: 1-1                        [400, 256, 6, 6]          --
│    └─Conv2d: 2-1                       [400, 64, 56, 56]         23,296
│    └─ReLU: 2-2                         [400, 64, 56, 56]         --
│    └─MaxPool2d: 2-3                    [400, 64, 27, 27]         --
│    └─Conv2d: 2-4                       [400, 192, 27, 27]        307,392
│    └─ReLU: 2-5                         [400, 192, 27, 27]        --
│    └─MaxPool2d: 2-6                    [400, 192, 13, 13]        --
│    └─Conv2d: 2-7                       [400, 384, 13, 13]        663,936
│    └─ReLU: 2-8                         [400, 384, 13, 13]        --
│    └─Conv2d: 2-9                       [400, 256, 13, 13]        884,992
│    └─ReLU: 2-10                        [400, 256, 13, 13]        --
│    └─Conv2d: 2-11                      [400, 256, 13, 13]       